In [1]:
import numpy as np
import scipy as sp

import pandas as pd

import matplotlib.pyplot as plt
pd.plotting.register_matplotlib_converters()
pd.options.mode.chained_assignment = None # suppress chained assignment warning

# Machine learning
from sklearn.model_selection import train_test_split as skl_ttsplit
import sklearn.decomposition as skl_dcmp
from sklearn.metrics import adjusted_rand_score

from pyclustering.cluster.kmedoids import kmedoids as KMedoids


import re

# Multiprocessing
from multiprocessing import Pool
import time

## Modify data directory here:

In [ ]:
# File name of files containing csvs of distance matrices, includes on label wildcard
# Attention: the numbers in this file are 1 - correlation
sp_distmat_wildcard = r'sp500/sp500_yearly_distmat_{}.csv'
# labels/years of labels of the time series of distance matrices
sp_labels = np.arange(2000, 2021)

statement_distmat_wildcard = r'financial_statement/distance_matrices_combined3/cluster_{}.csv'
statement_labels = np.arange(2000, 2021) 

hp_filename_wildcard = r'hp/hp_yearly_clusters_{}.csv'
hp_labels = np.arange(1988, 2020)


# Cluster sizes to process data for
n_clusters_range = np.array([x for x in range(10, 100, 10)] + [x for x in range(100, 241, 20)])


# Where to read cluster data from
cluster_wildcard = r'clusters/yearly_clusters_{}.csv'

## Getting Data

In [ ]:
company_data = pd.read_csv('company_data.csv')
company_data_convert_dict = {'ggroup': 'Int32', 'gind': 'Int32', 'gsector': 'Int32', 'gsubind': 'Int32', 'naics': 'Int32', 'sic': 'Int32', 'spcindcd': 'Int32', 'spcseccd': 'Int32'}
company_data = company_data.astype(company_data_convert_dict)
tic_lookup_by_gvkey = company_data.set_index('gvkey')[['tic']].to_dict()['tic']
gvkey_lookup_by_tic = company_data.set_index('tic')[['gvkey']].to_dict()['gvkey']

yearly_sp_distmat = {}
yearly_statement_distmat = {}
yearly_hp_clusters = {}
yearly_hp_cluster_labels = {}

yearly_available_companies = {} # list of companies for which data was available in each year

if sp_distmat_wildcard is not None:
    for year in sp_labels:
        yearly_sp_distmat[year] = pd.read_csv(sp_distmat_wildcard.format(year), index_col = 0)
        yearly_available_companies[year] = yearly_sp_distmat[year].columns.to_list()

if statement_distmat_wildcard is not None:
    for year in statement_labels:
        yearly_statement_distmat[year] = pd.read_csv(statement_distmat_wildcard.format(year), index_col = 0)
        yearly_available_companies[year] += yearly_statement_distmat[year].columns.to_list()
        yearly_available_companies[year] = list(dict.fromkeys(yearly_available_companies[year])) 

for year in hp_labels:
    yearly_hp_cluster_labels[year] = pd.read_csv(hp_filename_wildcard.format(year))
    yearly_hp_clusters[year] = {}
    for label in yearly_hp_cluster_labels[year]['cluster_label'].unique():
        yearly_hp_clusters[year][label] = yearly_hp_cluster_labels[year][yearly_hp_cluster_labels[year]['cluster_label'] == label]['tic'].to_list()

## Clustering

In [4]:
"""
Execute k-medoids for a pandas distance matrix.
Returns: tuple of clusters, medoids. If return_idx, returns indices rather than pandas columns.
"""
def kmedoids_clustering(distance_matrix, n_clusters, return_idx = False):
    kmedoids = KMedoids(
        distance_matrix.to_numpy(),
        initial_index_medoids = np.random.choice(len(distance_matrix), size=n_clusters, replace=True), 
        data_type = 'distance_matrix',
        tolerance = 1e-5,
        itermax = 1000,
    )
    kmedoids.process()
    
    kmedoids_medoids_idx = kmedoids.get_medoids()
    kmedoids_medoids = distance_matrix.columns[kmedoids_medoids_idx].to_list()
    kmedoids_clusters_idx = kmedoids.get_clusters()
    kmedoids_clusters = []
    for cluster in kmedoids_clusters_idx:
        kmedoids_clusters.append(distance_matrix.columns[cluster].to_list())
    
    return (kmedoids_clusters_idx, kmedoids_medoids_idx) if return_idx else (kmedoids_clusters, kmedoids_medoids)

**GIC**

In [5]:
# We only use ggroup and ggroup
GIC_codes = company_data[['tic', 'gsector', 'ggroup']]
GIC_codes.set_index('tic', inplace=True)

**NAICS**

In [6]:
NAICS_codes = company_data[['tic', 'naics']]
NAICS_codes['naics_2'] = NAICS_codes['naics'].apply(lambda s: int('{:04d}'.format(s)[0:2]) if not np.isnan(s) else np.NaN)
NAICS_codes['naics_3'] = NAICS_codes['naics'].apply(lambda s: int('{:04d}'.format(s)[0:3]) if not np.isnan(s) else np.NaN)
NAICS_codes['naics_4'] = NAICS_codes['naics'].apply(lambda s: int('{:04d}'.format(s)[0:4]) if not np.isnan(s) else np.NaN)
NAICS_codes.set_index('tic', inplace=True)

**SIC**

Attention: SIC 99 is a separate cluster! (https://mckimmoncenter.ncsu.edu/2digitsiccodes/)

In [7]:
SIC_codes = company_data[['tic', 'conm', 'sic']]
SIC_codes['sic_1'] = SIC_codes['sic'].apply(lambda s: (int('{:04d}'.format(s)[0]) if s//100 != 99 else 10) if not np.isnan(s) else np.NaN)
SIC_codes['sic_2'] = SIC_codes['sic'].apply(lambda s: int('{:04d}'.format(s)[0:2]) if not np.isnan(s) else np.NaN)

yearly_SIC_clusters_1 = {}
yearly_SIC_clusters_2 = {}
for year in yearly_available_companies:
    yearly_SIC_clusters_1[year] = {}
    yearly_SIC_clusters_2[year] = {}
    for sic_1 in SIC_codes['sic_1'].unique():
        yearly_SIC_clusters_1[year][sic_1] = SIC_codes[SIC_codes['sic_1'] == sic_1]['tic'].to_list()
    for sic_2 in SIC_codes['sic_2'].unique():
        yearly_SIC_clusters_2[year][sic_2] = SIC_codes[SIC_codes['sic_2'] == sic_2]['tic'].to_list()

**SP Industry Sector / Economic Sector Code**

In [8]:
SPC_codes = company_data[['tic', 'spcseccd', 'spcindcd']]
SPC_codes.set_index('tic', inplace=True)

**SP500**

In [9]:
class __clustering_wrapper:
    def __init__(self, n, yearly_input):
        self.n = n
        self.yearly_input = yearly_input
    def __call__(self, year):
        return kmedoids_clustering(self.yearly_input[year], self.n)

def clustering(yearly_input, n_clusters_range, threads = 5):
    clusters = {}
    medoids = {}
    for n in n_clusters_range:
        print('Clustering for n = {} ( '.format(n), end ='')
        clusters[n] = {}
        medoids[n] = {}
        
        if threads > 1:
            # Multiprocessing
            years = list(yearly_input.keys())
            with Pool(threads) as pool:
                result = pool.map(__clustering_wrapper(n, yearly_input), years)
            for i, year in enumerate(years):
                clusters[n][year] = result[i][0]
                medoids[n][year] = result[i][1]
            print('multiproc done ', end = '')
        
        else:
            # Without Multiprocessing
            for year in yearly_input:
                print('{} '.format(year), end = '')
                clusters[n][year], medoids[n][year] = kmedoids_clustering(yearly_input[year], n)
        print(')')
    return clusters, medoids
    

**KMedoids**

In [ ]:
start = time.time()
yearly_sp_clusters, yearly_sp_medoids = clustering(yearly_sp_distmat, n_clusters_range)
print('Elapsed while clustering SP500: {:.2f}'.format(time.time() - start))

In [ ]:
start = time.time()
yearly_statement_clusters, yearly_statement_medoids = clustering(yearly_statement_distmat, n_clusters_range, threads=6)
print('Elapsed while clustering statements: {:.2f}'.format(time.time() - start))

## Transformation into Monolithic Data Frame

In [ ]:
columns = ['sic_1', 'sic_2', 'naics_2', 'naics_3', 'naics_4', 'gsector', 'ggroup', 'spcseccd', 'spcindcd', 'hp']
columns += ['sp_{}'.format(n) for n in n_clusters_range]
columns += ['st_{}'.format(n) for n in n_clusters_range]
columns += ['ismedoid_sp_{}'.format(n) for n in n_clusters_range]
columns += ['ismedoid_st_{}'.format(n) for n in n_clusters_range]
cluster_dataframes = {}

for year in np.arange(2000, 2021):
    dataframe = pd.DataFrame(columns=columns, index = yearly_available_companies[year])
    
    dataframe['sic_1'] = SIC_codes.set_index('tic')['sic_1'].fillna(-1).astype('int')
    dataframe['sic_2'] = SIC_codes.set_index('tic')['sic_2'].fillna(-1).astype('int')
    
    dataframe['naics_2'] = NAICS_codes['naics_2'].fillna(-1).astype('int')
    dataframe['naics_3'] = NAICS_codes['naics_3'].fillna(-1).astype('int')
    dataframe['naics_4'] = NAICS_codes['naics_4'].fillna(-1).astype('int')
    
    dataframe['gsector'] = GIC_codes['gsector'].fillna(-1).astype('int')
    dataframe['ggroup'] = GIC_codes['ggroup'].fillna(-1).astype('int')
    
    dataframe['spcseccd'] = SPC_codes['spcseccd'].fillna(-1).astype('int')
    dataframe['spcindcd'] = SPC_codes['spcindcd'].fillna(-1).astype('int')
    
    if year in yearly_hp_cluster_labels:
        dataframe['hp'] = yearly_hp_cluster_labels[year].set_index('tic')['cluster_label'].fillna(-1).astype('int')
        dataframe['hp'] = dataframe['hp'].fillna(-1).astype('int')
    
    # Transform k-Medoids cluster data format for tic lists to a label for each tic
    for n in n_clusters_range:
        if year in yearly_sp_clusters[n]:
            # write cluster labels
            for i, cluster in enumerate(yearly_sp_clusters[n][year]):
                dataframe.at[cluster, 'sp_{}'.format(n)] = i
            # mark medoids
            dataframe.at[yearly_sp_medoids[n][year], 'ismedoid_sp_{}'.format(n)] = 1
            dataframe['ismedoid_sp_{}'.format(n)].fillna(0, inplace=True)
            
        if year in yearly_statement_clusters[n]:
            # write cluster labels
            for i, cluster in enumerate(yearly_statement_clusters[n][year]):
                dataframe.at[cluster, 'st_{}'.format(n)] = i
            # mark medoids
            dataframe.at[yearly_statement_medoids[n][year], 'ismedoid_st_{}'.format(n)] = 1
            dataframe['ismedoid_st_{}'.format(n)].fillna(0, inplace=True)
            
    cluster_dataframes[year] = dataframe

In [26]:
# Write output
for year in cluster_dataframes:
    cluster_dataframes[year].to_csv(cluster_wildcard.format(year))